In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!nvidia-smi
!pip install ultralytics
!pip install roboflow

Sun Apr 27 21:14:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="oJjnkc4a4xtkt2sBKZYi")
project = rf.workspace("roboflow-gw7yv").project("self-driving-car")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Self-Driving-Car-3 in yolov8:: 100%|██████████| 59606/59606 [00:06<00:00, 9029.52it/s] 


In [5]:
!ls /kaggle/working

__notebook__.ipynb  Self-Driving-Car-3


In [6]:
import os
import shutil

# Create directories
os.makedirs('/kaggle/working/Self-Driving-Car-3/train/images', exist_ok=True)
os.makedirs('/kaggle/working/Self-Driving-Car-3/train/labels', exist_ok=True)
os.makedirs('/kaggle/working/Self-Driving-Car-3/valid/images', exist_ok=True)
os.makedirs('/kaggle/working/Self-Driving-Car-3/valid/labels', exist_ok=True)

# Paths
images_path = '/kaggle/working/Self-Driving-Car-3/export/images'
labels_path = '/kaggle/working/Self-Driving-Car-3/export/labels'

# List all images
all_images = os.listdir(images_path)

# 80-20 split
split_idx = int(0.8 * len(all_images))
train_images = all_images[:split_idx]
valid_images = all_images[split_idx:]

# Copy files
for img in train_images:
    shutil.copy(os.path.join(images_path, img), '/kaggle/working/Self-Driving-Car-3/train/images')
    label_file = img.replace('.jpg', '.txt').replace('.png', '.txt')
    shutil.copy(os.path.join(labels_path, label_file), '/kaggle/working/Self-Driving-Car-3/train/labels')

for img in valid_images:
    shutil.copy(os.path.join(images_path, img), '/kaggle/working/Self-Driving-Car-3/valid/images')
    label_file = img.replace('.jpg', '.txt').replace('.png', '.txt')
    shutil.copy(os.path.join(labels_path, label_file), '/kaggle/working/Self-Driving-Car-3/valid/labels')

print("✅ Train/Valid split done!")

✅ Train/Valid split done!


In [7]:
!yolo task=detect mode=train model=yolov8m.pt data=/kaggle/working/Self-Driving-Car-3/data.yaml epochs=10

100%|███████████████████████████████████████| 49.7M/49.7M [00:00<00:00, 215MB/s]
Ultralytics 8.3.118 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/kaggle/working/Self-Driving-Car-3/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fals